성모병원 ICU로부터 수집한 PPG-RR 데이터를 이용해 7가지 모델을 검증하자.

In [1]:
import os
import numpy as np
import pandas as pd
import multiprocessing
from scipy import signal
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from model_src.DilatedResNet import DilatedResNet
from model_src.BianResnet import BianResNet
from model_src.LSTM import VanillaLSTM, CNNLSTM, BiLSTM, BiLSTMAttn
from model_src.RespNet import RespNet

print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
DATA_PATH = '/root/Workspace/DataLake/stMary'
DATA_SAVE_PATH = '/root/Workspace/Project-RRpo-2ndStudy/dataset' 

2023-10-26 21:40:14.748525: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 21:40:14.800301: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 21:40:15.562290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def interpolation(x, input):
    x0 = int(np.floor(x))
    y0 = input[x0]
    x1 = int(np.ceil(x))
    y1 = input[x1]
    y = (y1-y0)*(x-x0) + y0
    return y


def signal_resample(input_signal, org_fs, new_fs, method='interpolation'):
    output_signal = []
    new_x = np.arange(0, len(input_signal), org_fs/new_fs)
    
    if method == 'interpolation': 
        interp = interpolation

    for x in new_x:
        y = interp(x, input_signal)
        output_signal.append(y)

    return np.asarray(output_signal)


def generate_dataset(arg_pleths, arg_resps, fs=125, shift_factor=4):
    """
    성모병원에서 수집된 데이터의 특성상 이러한 전처리를 진행해주어야 한다.
    """
    import copy
    dataset = []
    window_size = fs * 60 # 7500
    shift = int(window_size/shift_factor)
    samples_len = len(arg_pleths)

    cpy_resps = copy.deepcopy(arg_resps)
    cpy_pleths = copy.deepcopy(arg_pleths)

    for j in range(samples_len):
        rr = cpy_resps[j]; ppg = cpy_pleths[j]

        rr['offset'] = (rr['offset']-rr['offset'].min())/1000
        size_lim = int(fs * np.ceil(rr['offset'].max()))
        ppg = ppg[:size_lim]
        shift_n_times = int((len(ppg)-window_size)/shift)+1

        samp_rr = [len(rr.loc[ (rr['offset']>=0+(int(shift/fs)*i)) & ((rr['offset']<int(window_size/fs)+(int(shift/fs)*i))) ]) for i in range(shift_n_times)]
        samp_ppg = [ppg[0+(shift*i):window_size+(shift*i)] for i in range(shift_n_times)]

        for i in range(len(samp_ppg)):
            temp = []
            temp.append(samp_ppg[i])
            temp.append(samp_rr[i])
            dataset.append(temp)

    return dataset


def preprocessing(targets, numtaps, cutoff, shift_factor, org_fs, new_fs, processes):
    print('Extract PLETH/RESP')
    pleths = [pd.read_csv(f'{DATA_PATH}/{sid}/pleth.csv', header=None, names=['sid', 'offset', 'pleth']).pleth.values for sid in targets.id.unique()]
    resps = [pd.read_csv(f'{DATA_PATH}/{sid}/respirationTimeline.csv', header=None, names=['sid', 'offset']) for sid in targets.id.unique()]

    # Before filtering: Check NaN
    for pleth in pleths:
        if any(np.isnan(pleth)):
            print('check')

    # Before filtering: Convert type as np.float32
    pleths = list(map(lambda pleth: pleth.astype(np.float32), pleths))


    print('Init Preprocessing: Filtering')
    taps = signal.firwin(numtaps=numtaps, cutoff=cutoff, window='hamming', pass_zero=False, fs=org_fs)
    pool = multiprocessing.Pool(processes=processes)
    filtered_pleths = pool.starmap(signal.filtfilt, [(taps, 1.0, pleth) for pleth in pleths])
    pool.close()
    pool.join()


    print('Init Preprocessing: Windowing')
    dataset = generate_dataset(filtered_pleths, resps, shift_factor=shift_factor)


    print('Init Preprocessing: Resampling')
    pool = multiprocessing.Pool(processes=processes)
    result = pool.starmap(signal_resample, [(pleth[0], org_fs, new_fs) for pleth in dataset])
    pool.close()
    pool.join()

    new_patient = []
    for i in range(len(dataset)):
        temp = []
        temp.append(result[i])
        temp.append(dataset[i][1])
        new_patient.append(temp)

    return new_patient


def prepare_modeling(dataset=None, batchsize=None):
    print(f'Prepare modeling')
    pleths = []
    resps = []
    for ppg, rr in dataset:
        pleths.append(ppg.astype(np.float32))
        resps.append(rr)
    pleths = np.asarray(pleths)
    resps = np.asarray(resps)
    print(pleths.shape, resps.shape)

    scaler = MinMaxScaler()
    scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
    print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

    x, y = scaled_pleths[:], resps[:]

    return tf.data.Dataset.from_tensor_slices((x, y)).batch(batchsize)

In [3]:
subjects = pd.read_csv(f'{DATA_PATH}/patients.csv')
patients = subjects.loc[subjects['diagnosis']!='0']
rnd_patients = patients.sample(frac=1, random_state=42)
kf = KFold(n_splits=5)

In [4]:
ps_dataset = preprocessing(rnd_patients, numtaps=2000, cutoff=[0.1, 0.4], shift_factor=60, org_fs=125, new_fs=30, processes=40)
np.save(f'{DATA_SAVE_PATH}/230920/stmary-preprocessed.npy', np.array(ps_dataset))

Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling


AttributeError: 'list' object has no attribute 'shape'

In [15]:
ps_dataset = np.load(f'{DATA_SAVE_PATH}/230920/stmary-preprocessed.npy', allow_pickle=True)

In [12]:
prepare_modeling(ps_dataset, batchsize=256)

Prepare modeling
(6508, 1800) (6508,)
(6508, 1800, 1) <class 'numpy.float32'>


<_BatchDataset element_spec=(TensorSpec(shape=(None, 1800, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [6]:
counter = 0
cutoff = [0.1, 0.4]
raw_dataset = {
    'train': [],
    'val': []
}
dataset = {
    'train': [],
    'val': []
}

In [ ]:
for train_idx, val_idx in kf.split(train_patients):
    print(f'{counter}th K-fold')
    counter = counter + 1
    # 이렇게 하는 이유는 Train과 Validation을 완벽히 구별시키기 위함이다.
    X_train = preprocessing(train_patients.iloc[train_idx], numtaps=2000, cutoff=cutoff, shift_factor=60, org_fs=125, new_fs=30, processes=40)
    X_val = preprocessing(train_patients.iloc[val_idx], numtaps=2000, cutoff=cutoff, shift_factor=60, org_fs=125, new_fs=30, processes=40)
    print(f'Preprocessing finished: {len(X_train)} / {len(X_val)}')

    raw_dataset['train'].append(X_train)
    raw_dataset['val'].append(X_val)
    
    # train_dataset = prepare_modeling(X_train, batchsize=256)
    # val_dataset = prepare_Mmodeling(X_val, batchsize=256)

    # dataset['train'].append(train_dataset)
    # dataset['val'].append(val_dataset)

    np.save(f'{DATA_SAVE_PATH}/230919_filt{cutoff[0]}to{cutoff[1]}_win60s_rsamp30_{counter}Fold_train.npy', X_train)
    np.save(f'{DATA_SAVE_PATH}/230919_filt{cutoff[0]}to{cutoff[1]}_win60s_rsamp30_{counter}Fold_val.npy', X_val)

In [ ]:
for i in range(5):
    X_train = np.load(f'{DATA_SAVE_PATH}/230919_filt0.1to0.4_win60s_rsamp30_{counter+2}Fold_train.npy', allow_pickle=True)
    X_val = np.load(f'{DATA_SAVE_PATH}/230919_filt0.1to0.4_win60s_rsamp30_{counter+2}Fold_val.npy', allow_pickle=True)
    
    train_dataset = prepare_modeling(X_train, batchsize=256)
    val_dataset = prepare_modeling(X_val, batchsize=256)
    
    dataset['train'].append(train_dataset)
    dataset['val'].append(val_dataset)

In [ ]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
callbacks = [
    EarlyStopping(monitor='val_loss', patience=33),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [ ]:
models = [
    BianResNet(), 
    # DilatedResNet(num_of_blocks=3, dwn_kernel_size=3, filters=32),
    # RespNet(),
    # VanillaLSTM(),
    # CNNLSTM(),
    # BiLSTM(),
    # BiLSTMAttn(units=64, units_attn=64, dropout=0.5)
]

In [ ]:
# MODELS = ['Bian', 'DilatedResNet', 'RespNet', 'LSTM', 'CNNLSTM', 'BiLSTM', 'BiLSTMAttn']
# MODELS = ['CNNLSTM', 'BiLSTM', 'BiLSTMAttn']
MODELS = ['Bian']
train_losses = []
val_losses = []

for i, model in enumerate(models):
    print(f'\n{MODELS[i]}')
    print('====='*20)
    tmp_train_losses = []
    tmp_val_losses = []
    
    for j in range(5):
        print(f'{j+1}th K-fold')

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
            loss=keras.losses.MeanAbsoluteError(),
            metrics=keras.metrics.MeanAbsoluteError()
        )  

        callbacks.append(ModelCheckpoint(f'../models/230920/{MODELS[i]}-230919dataset-KF{j+1}', monitor='val_loss', save_best_only=True))
        
        history = model.fit(
            dataset['train'][j],
            epochs=EPOCHS,
            callbacks=callbacks,
            validation_data=dataset['val'][j]
        )

        val_loss_min_ix = np.argmin(history.history['val_loss'])
        tmp_train_losses.append(history.history['loss'][val_loss_min_ix])
        tmp_val_losses.append(history.history['val_loss'][val_loss_min_ix])
    
    train_losses.append(np.asarray(tmp_train_losses))
    val_losses.append(np.asarray(tmp_val_losses))
    print(f'{np.mean(train_losses)} ± {np.std(train_losses)}')
    print(f'{np.mean(val_losses)} ± {np.std(val_losses)}')

테스트 해야 할 내용:
- cutoff [0.1~0.4] 에서의 결과를 확인하고 이에 따른 논문 수정이 필요하다. 이럴 경우 Result까지 수정할 필요가 있다. 이에 대해 아래 레퍼런스들을 참고하여

    + IQBAL, T., A. ELAHI, S. GANLY, W. WIJNS, et al. Photoplethysmography-Based Respiratory Rate Estimation Algorithm for Health Monitoring Applications. Journal of Medical and Biological Engineering, 2022/04/01 2022, 42(2), 242-252. [0.1-0.4]

- 호흡수가 정상호흡(12~18brpm), 빠른호흡(>18brpm), 느린호흡(<12brpm) 각각에 대해서 예측 RR과 실제 RR 사이의 차이를 MAE, 백분율로 표시할 것 권장.

    + 이 세 부류의 호흡에 대해서 Figure를 보여줄 것 권장.
 

In [ ]:
model_bian = [tf.keras.models.load_model(f'../models/230920/Bian-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
model_bian[1].evaluate(dataset['train'][0])

In [ ]:
for i in range(5):
    

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_bian[i].evaluate(dataset['train'][i]))
    val_res.append(model_bian[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')

In [ ]:
model_dil = [tf.keras.models.load_model(f'../models/230919-DilatedResNet-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_dil[i].evaluate(dataset['train'][i]))
    val_res.append(model_dil[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')

In [ ]:
model_unet = [tf.keras.models.load_model(f'../models/230919-RespNet-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_unet[i].evaluate(dataset['train'][i]))
    val_res.append(model_unet[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')

In [ ]:
model_lstm = [tf.keras.models.load_model(f'../models/230919-LSTM-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_lstm[i].evaluate(dataset['train'][i]))
    val_res.append(model_lstm[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')

In [ ]:
model_cnnlstm = [tf.keras.models.load_model(f'../models/230919-CNNLSTM-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_cnnlstm[i].evaluate(dataset['train'][i]))
    val_res.append(model_cnnlstm[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')

In [ ]:
mae = []; std = []
for i in range(5):
    y_pred = []; y_true = [];
    for x,y in dataset['train'][i].as_numpy_iterator():
        y_pred.append(model_cnnlstm[0].predict(x))
        y_true.append(y)
    
    y_pred2 = np.concatenate(y_pred, axis=0)
    y_true2 = np.concatenate(y_true).reshape(-1,1)

    mae.append(np.mean(np.abs(y_true2 - y_pred2)))
    std.append(np.std(np.abs(y_true2 - y_pred2)))

In [ ]:
print(mae)
print(std)

In [ ]:
model_bilstm = [tf.keras.models.load_model(f'../models/230919-BiLSTM-230919dataset-KF{i+1}') for i in range(5)]

In [ ]:
train_res = []; val_res = []
for i in range(5):
    train_res.append(model_bilstm[i].evaluate(dataset['train'][i]))
    val_res.append(model_bilstm[i].evaluate(dataset['val'][i]))
print(f'TRAIN: {np.mean(train_res)}±{np.std(train_res)} / VAL: {np.mean(val_res)}±{np.std(val_res)}')